In [1]:
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd

from os import listdir
from os.path import isfile, join

import gensim
import nltk
from nltk.corpus import brown

from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

import transformers

import seaborn as sns
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import math

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import SpatialDropout1D


# Gerar Dataset

In [155]:
mypath = 'E:/Code/prf-recursos-multas/outputs'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

outfile = 'E:/Code/prf-recursos-multas/dataset/multas.csv'

sepc = '`'

In [16]:
with open(outfile, 'a', encoding='utf-8') as file:
    file.write('arquivo' + sepc + 'parecer' + sepc + 'cod_multa' + sepc + 'texto\n')
    file.close()

for f in onlyfiles:
    fpath = mypath + '/' + f
    #print(fpath)
    line = ''
    
    x = f.split('+')
    if (len(x) != 3):
        continue
    for e in x:
        line += e + sepc
    
    
    

    with open(fpath, 'r', encoding='utf-8') as file:
        fstr = file.read().replace('\n', '%EOL%').replace(sepc, '')
        line += fstr + "\n"
        file.close()

    #print(file)    
        
    with open(outfile, 'a', encoding='utf-8') as file:
        file.write(line)
        file.close()
        


# Carregar Dataset

In [2]:
sepc = '`'

df_path_1 = 'E:/Code/prf-recursos-multas/datasets/multas.csv'
df_path_2 = 'E:/Code/prf-recursos-multas/datasets/multas_v2.csv'

df1 = pd.read_csv(df_path_1, sep=sepc, encoding='utf-8')
df2 = pd.read_csv(df_path_2, sep=sepc, encoding='utf-8')

def remove_txt(row):  
    return row['cod_multa'].replace('.txt','')
df1['cod_multa'] = df1.apply(remove_txt, axis=1)

In [3]:
df = pd.concat([df1,df2])

In [16]:
df

,arquivo,parecer,cod_multa,texto
0,processo-08664010983201828_text,I,50610,"Eu, acima identificado venho por meio deste fo..."
1,processo-08664011015201839_text,I,59670,ILMO SENHOR SUPERINTENDENTE DA POLICIA RODOVIA...
2,processo-08664011027201863_text,I,54282,"ILUSTRISSIMO SENHOR, PRESIDENTE DA JARI, DO De..."
3,processo-08664011030201887_text,I,54521,JOZIVAN HELIO DE ARAUJO vem tempestivamente e ...
4,processo-08664011031201821_text,I,54521,JOZIVAN HELIO DE ARAUJO vem tempestivamente e ...
...,...,...,...,...
151,SEI_08664.012658_2020_14,I,58194,%EOL%%EOL%MINISTERIO DA JUSTICA%EOL%DEPARTAME...
152,SEI_08664.012690_2020_08,I,59670,%EOL%%EOL%ILUSTRISSIMO SENHOR SUPERINTENDENTE...
153,SEI_08664.012699_2020_19,I,50100,%EOL%%EOL% %EOL%%EOL% %EOL%%EOL%EXCELENTISSIM...
154,SEI_08664.012707_2020_19,I,51691,%EOL%%EOL%. REQUERIMENTO DE DEFESA PREVIA%EO...


In [24]:
vocab = set([])

for x in df["texto"]:
    for y in x.split(" "):
        if (len(y) > 0 and y[0] != '%'):
            vocab.add(y)

len(vocab)

262152

# Pré-processamento

## Lowercasing

In [25]:
df.texto = df.texto.str.replace('%EOL%','\n')

In [26]:
df.texto = df.texto.str.lower()

## Remoção de caracteres inválidos

In [27]:
alphabet = "abcdefghijklmnopqrstuvxzywç" + "àáâãéêíóõôú"
other = "\n" + " " + "\t" + ".,?!:;"

valid_characters = list(alphabet + other)

In [28]:
def clean_text_characters(row):
    txt = ''
    
    al = list(alphabet)
    ot = list(other)
    
    for c in row.texto:
        if (str(c) in ot):
            txt += ' '
        elif (str(c) in al):
            txt += c
    
    row.text = txt
    return txt
df["texto2"] = df.apply(clean_text_characters, axis=1)

## Remoção das stopwords

Link útil: http://www.nltk.org/howto/portuguese_en.html

Caso necessáiro, mais stopwords: https://gist.github.com/alopes/5358189

In [29]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download()

In [30]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[0:10]

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

In [31]:
stopwords += ".,?!:;"

In [32]:
def remove_stop_words(row):
    txt = []
    #print(type(row.texto2))
    tokens = nltk.word_tokenize(row.texto2)
    for t in tokens:
        if (not t in stopwords):
            txt.append(t)
    
    return txt

In [33]:
df["texto3"] = df.apply(remove_stop_words, axis=1)

## Stemming

In [35]:
stemmer = nltk.stem.RSLPStemmer()
def stemming(row):
    txt = []
    #print(type(row.texto2))
    #print(len(df.texto3))
    for t in row.texto3:
        txt.append(stemmer.stem(t))
    
    return txt

In [36]:
df["texto4"] = df.apply(stemming,axis=1)

In [39]:
vocab = set([])

for x in df["texto4"]:
    for y in x:
        vocab.add(y)

len(vocab)

63580

In [ ]:
df.to_csv('E:/Code/prf-recursos-multas/dataset/multas_v3.csv', index=False)

# Separando base

In [2]:
df_path = 'E:/Code/prf-recursos-multas/datasets/multas_v3.csv'

df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

## Meio a meio

In [3]:
dfi = df[ df['parecer'] == 'I' ][0:173]
dfd = df[ df['parecer'] == 'D' ]

In [4]:
df2 = pd.concat([dfi,dfd])
df = df2.sample(frac=1, random_state=1).reset_index(drop=True)

## Dobro de indeferimento

In [20]:
dfi = df[ df['parecer'] == 'I' ][0:173*2]
dfd = df[ df['parecer'] == 'D' ]
df2 = pd.concat([dfi,dfd])
df = df2.sample(frac=1, random_state=1).reset_index(drop=True)

# Separando dataset (80% treino & 20% teste)

In [15]:
df = df.sample(frac=1, random_state=1).reset_index(drop=True)

train_num = int(df.shape[0]*0.8)
test_num = int(df.shape[0]*0.2)

# Tokenização

In [29]:
# calcula os codigos de multa mais relevantes

freq2 = {}

qtd = 0

multa_qtd = 0

for index, row in df.iterrows():
    if not row.cod_multa in freq2:
        freq2[row.cod_multa] = 1
    else:
        freq2[row.cod_multa] += 1

for k in freq2.items():
    if (k[1] >= qtd):
        multa_qtd += k[1]

# calcula a qtd de palavras totais

freq3 = {}

qtd = 0

for index, row in df.iterrows():
    if freq2[row.cod_multa] >= qtd:
        for x in row.texto4:
            
            if not x in freq3:
                freq3[x] = 1

## Bow

In [30]:
cols = ['arquivo', 'parecer', 'cod_multa']
for x in freq3.items():
    cols.append(x[0])

In [31]:
def list_of_zeros(qtd):
    l = []
    for x in range(0,qtd):
        l.append(0)
    return l

In [32]:
zeros = list_of_zeros(len(cols)-3)

bow_rows = []

for index, row in df.iterrows():
    if freq2[row.cod_multa] >= qtd:
        r = [row.arquivo, row.parecer, row.cod_multa] + zeros
        bow_rows.append(r)
        
        

In [33]:
df_bow = pd.DataFrame (bow_rows, columns = cols)

In [34]:
aux = 0
for index, row in df.iterrows():
    if freq2[row.cod_multa] >= qtd:
        for x in row.texto4:
            df_bow.loc[aux,x] += 1
        aux += 1

In [35]:
df_bow = df_bow[ (df_bow.parecer == 'D') | (df_bow.parecer == 'I')]

In [36]:
cod_to_int = {}

aux = 0

for index, row in df.iterrows():
    if freq2[row.cod_multa] >= qtd:
        if not row.cod_multa in cod_to_int:
            cod_to_int[row.cod_multa] = aux
            aux += 1

In [37]:
def convert_cod_to_int(row):
    return cod_to_int[row.cod_multa]

df_bow['cod_multa'] = df_bow.apply(convert_cod_to_int, axis=1)

In [38]:
def convert_par_to_int(row):
    if (row.parecer == 'D'):
        return 1
    else:
        return 0

df_bow['parecer'] = df_bow.apply(convert_par_to_int, axis=1)

In [39]:
df_bow

,arquivo,parecer,cod_multa,ministeri,justig,departament,polic,rodovi,feder,superintendenc,...,infragaoq,sider,ovou,lentida,injustigas,infragdonenhumaprovatestemunh,daadministraca,constatacgd,aorecorrerapresent,wlegit
0,SEI_08664.004601_2018_27_text,0,0,1,1,1,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
1,SEI_08664.006167_2018_10_text,0,1,0,0,1,2,2,6,0,...,0,0,0,0,0,0,0,0,0,0
2,SEI_08664.006401_2018_17_text,0,2,0,0,0,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,SEI_08664011448201894_text,1,3,6,0,3,15,11,14,3,...,0,0,0,0,0,0,0,0,0,0
4,SEI_08664.003985_2018_61_text,0,4,0,0,1,1,1,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,SEI_08664.000255_2018_16_text,0,49,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
515,SEI_08664011242201864_text,0,3,5,1,4,14,11,13,2,...,0,0,0,0,0,0,0,0,0,0
516,SEI_08664010373201824_text,0,17,7,0,3,14,12,15,3,...,0,0,0,0,0,0,0,0,0,0
517,SEI_08650.008641_2021_01,0,9,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#df_bow.to_csv('E:/Code/prf-recursos-multas/dataset/multas_bow_3.csv', index=False)

# Classificação

## TF-IDF

In [41]:
df_tfidf = df_bow.copy()

df_bow_target_parecer = df_tfidf['parecer']

df_tfidf = df_tfidf.drop(['arquivo', 'parecer', 'cod_multa'],axis=1)
df_tfidf *= 1.0
df_tfidf

,ministeri,justig,departament,polic,rodovi,feder,superintendenc,regionalrn,requer,pedr,...,infragaoq,sider,ovou,lentida,injustigas,infragdonenhumaprovatestemunh,daadministraca,constatacgd,aorecorrerapresent,wlegit
0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,2.0,2.0,6.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,0.0,3.0,15.0,11.0,14.0,3.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515,5.0,1.0,4.0,14.0,11.0,13.0,2.0,0.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516,7.0,0.0,3.0,14.0,12.0,15.0,3.0,0.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
517,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
M = df_tfidf.shape[0] * 1.0
aux = df_tfidf.columns


dft = {}
for w in aux:
    x = df_tfidf[df_tfidf[w] > 0].shape[0]
    if (x == 0):
        x = 1.0
    dft[w] = math.log2(M/x)

In [43]:
for index, row in df_tfidf.iterrows():
    cnt = 0
    for w in aux:
        if (row[w]>0.0):
            cnt += 1
    for w in aux:
        row[w] = (row[w]/cnt) * dft[w]

In [44]:
df_tfidf

,ministeri,justig,departament,polic,rodovi,feder,superintendenc,regionalrn,requer,pedr,...,infragaoq,sider,ovou,lentida,injustigas,infragdonenhumaprovatestemunh,daadministraca,constatacgd,aorecorrerapresent,wlegit
0,0.002190,0.004715,0.001451,0.000571,0.000767,0.001332,0.001991,0.008275,0.002817,0.007544,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.000000,0.000000,0.003077,0.002421,0.003255,0.005652,0.000000,0.000000,0.005974,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.002616,0.001759,0.001018,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.006002,0.000000,0.001988,0.003909,0.003855,0.002840,0.002728,0.000000,0.004502,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.001868,0.000735,0.000988,0.001143,0.000000,0.000000,0.000906,0.009711,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0.000000,0.000000,0.002702,0.000000,0.000000,0.000827,0.000000,0.000000,0.007869,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
515,0.007018,0.003022,0.003719,0.005120,0.005409,0.003700,0.002552,0.000000,0.007672,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
516,0.008239,0.000000,0.002339,0.004294,0.004949,0.003580,0.003210,0.000000,0.006433,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000548,0.000000,0.000000,0.003476,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [41]:
#df_tfidf.to_csv('E:/Code/prf-recursos-multas/dataset/multas_tfidf_3.csv', index=False)

## Classificação com TF-IDF (Por Deferimento)
### SVM

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_tfidf, df_bow_target_parecer, test_size=0.2,shuffle=False)

NameError: name 'df_tfidf' is not defined

In [46]:
clf = svm.SVC(kernel='linear', C=100)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8365384615384616


In [47]:
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.96      0.88        67
           1       0.88      0.62      0.73        37

    accuracy                           0.84       104
   macro avg       0.85      0.79      0.81       104
weighted avg       0.84      0.84      0.83       104



In [40]:
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision']

for score in scores:
    clf = GridSearchCV(SVC(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn

Best parameters set found on development set:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.252 (+/-0.007) for {'C': 1, 'kernel': 'linear'}
0.684 (+/-0.123) for {'C': 10, 'kernel': 'linear'}
0.723 (+/-0.035) for {'C': 100, 'kernel': 'linear'}
0.711 (+/-0.083) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.66      0.86      0.75        36
           1       0.78      0.53      0.63        34

    accuracy                           0.70        70
   macro avg       0.72      0.70      0.69        70
weighted avg       0.72      0.70      0.69        70




## LSTM

In [40]:
df_path = 'E:/Code/prf-recursos-multas/datasets/multas_v3.csv'

df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

In [41]:
df.parecer.value_counts()

I    960
D    173
A     13
N      2
Name: parecer, dtype: int64

In [42]:
df = df[ (df.parecer == 'D') | (df.parecer == 'I')]

In [43]:
def test(row):
    txt = ""
    for t in row["texto4"]:
        txt += t + " "
    return txt
df["texto5"] = df.apply(test,axis=1)

In [54]:

xxxx=200

tokenizer = Tokenizer(nb_words=xxxx, lower=True,split=' ')
tokenizer.fit_on_texts(df['texto5'].values)
#print(tokenizer.word_index)  # To see the dicstionary
X = tokenizer.texts_to_sequences(df['texto5'].values)
X = pad_sequences(X)

In [57]:
len(X[100])

3786

In [12]:


embed_dim = 128
lstm_out = 300
batch_size = 32

model = Sequential()
model.add(Embedding(xxxx, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(units=lstm_out, dropout = 0.1, recurrent_dropout = 0.1))
#model.add(LSTM(units=lstm_out)
##model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3395, 128)         25600     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 3395, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               514800    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
Y = pd.get_dummies(df['parecer']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  test_size=0.2, seed = 1, stratify=Y)
#X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  test_size=0.2,shuffle=False)

#Here we train the Network.

model.fit(X_train, Y_train)

9/9 [==============================] - 471s 52s/step - loss: 0.6782 - accuracy: 0.5615


In [ ]:
score,acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

In [ ]:
Y_pred = model.predict(X_valid)

In [ ]:
def binarize(x):
    rtn = []
    for i in x:
        if i[0] > i[1]:
            rtn.append(0)
        else:
            rtn.append(1)
    return rtn

In [ ]:
y_pred = binarize(Y_pred)
y_true = binarize(Y_valid)

In [ ]:
print(classification_report(y_true, y_pred))

In [31]:
df_path = 'E:/Code/prf-recursos-multas/dataset/multas_v3.csv'
df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

In [6]:
df.shape

(1148, 7)

In [32]:
df['parecer'].value_counts()

I    960
D    173
A     13
N      2
Name: parecer, dtype: int64

**Pesquisar por**

**split_test_train com o parametro stratify**

# Testes

## Definindo as funções

In [3]:
def list_of_zeros(qtd):
    l = []
    for x in range(0,qtd):
        l.append(0)
    return l

cod_to_int = {}

def convert_cod_to_int(row):
    global cod_to_int
    return cod_to_int[row.cod_multa]

def convert_par_to_int(row):
    if (row.parecer == 'D'):
        return 1
    else:
        return 0

In [4]:
def gerar_bow_df(df):
    
    freq2 = {}

    qtd = 0

    multa_qtd = 0

    for index, row in df.iterrows():
        if not row.cod_multa in freq2:
            freq2[row.cod_multa] = 1
        else:
            freq2[row.cod_multa] += 1

    for k in freq2.items():
        if (k[1] >= qtd):
            multa_qtd += k[1]

    # calcula a qtd de palavras totais

    freq3 = {}

    qtd = 0
    

    aux = 0

    for index, row in df.iterrows():
        if freq2[row.cod_multa] >= qtd:
            for x in row.texto4:

                if not x in freq3:
                    freq3[x] = 1

    cols = ['arquivo', 'parecer', 'cod_multa']
    for x in freq3.items():
        cols.append(x[0])

    zeros = list_of_zeros(len(cols)-3)

    bow_rows = []

    for index, row in df.iterrows():
        if freq2[row.cod_multa] >= qtd:
            r = [row.arquivo, row.parecer, row.cod_multa] + zeros
            bow_rows.append(r)

    df_bow = pd.DataFrame (bow_rows, columns = cols)
    aux = 0
    for index, row in df.iterrows():
        if freq2[row.cod_multa] >= qtd:
            for x in row.texto4:
                df_bow.loc[aux,x] += 1
            aux += 1
    global cod_to_int
    cod_to_int = {}

    aux = 0

    for index, row in df.iterrows():
        if freq2[row.cod_multa] >= qtd:
            if not row.cod_multa in cod_to_int:
                cod_to_int[row.cod_multa] = aux
                aux += 1
                
    df_bow['cod_multa'] = df_bow.apply(convert_cod_to_int, axis=1)
    df_bow['parecer'] = df_bow.apply(convert_par_to_int, axis=1)

    return df_bow

In [5]:
def gerar_tfidf_df(df_bow):
    df_tfidf = df_bow.copy()

    df_bow_target_parecer = df_tfidf['parecer']

    df_tfidf = df_tfidf.drop(['arquivo', 'parecer', 'cod_multa'],axis=1)
    df_tfidf *= 1.0
    df_tfidf

    M = df_tfidf.shape[0] * 1.0
    aux = df_tfidf.columns


    dft = {}
    for w in aux:
        x = df_tfidf[df_tfidf[w] > 0].shape[0]
        if (x == 0):
            x = 1.0
        dft[w] = math.log2(M/x)

    for index, row in df_tfidf.iterrows():
        cnt = 0
        for w in aux:
            if (row[w]>0.0):
                cnt += 1
        for w in aux:
            row[w] = (row[w]/cnt) * dft[w]

    return df_tfidf

In [6]:
def test(row):
    txt = ""
    for t in row["texto4"]:
        txt += t + " "
    return txt

def binarize(x):
    rtn = []
    for i in x:
        if i[0] > i[1]:
            rtn.append(0)
        else:
            rtn.append(1)
    return rtn

In [7]:
pl_loop = 20

def pipeline(df, df_bow, df_tfidf):
    df_bow_data = df_bow.drop(['arquivo', 'parecer', 'cod_multa'], axis=1)
    df_bow_target_multa = df_bow['cod_multa']
    df_bow_target_parecer = df_bow['parecer']
    #-------------------------------------
    print("Naive-bayes + BOW")
    

    for i in range (0,pl_loop):
        print(">>" + str(i))
        X_train, X_test, y_train, y_test = train_test_split(df_bow_data, df_bow_target_parecer, test_size=0.2)
        
        clf = MultinomialNB()

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
    
    
    #-------------------------------------
    print("SVM + BOW")
    for i in range (0,pl_loop):
        print(">>" + str(i))
        X_train, X_test, y_train, y_test = train_test_split(df_bow_data, df_bow_target_parecer, test_size=0.2)
        
        clf = svm.SVC(kernel='linear', C=100)

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
    
    #-------------------------------------
    
    print("Naive-bayes + TFIDF")
    for i in range (0,pl_loop):
        print(">>" + str(i))
        X_train, X_test, y_train, y_test = train_test_split(df_tfidf, df_bow_target_parecer, test_size=0.2)
        
        clf = MultinomialNB()

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
    
    #-------------------------------------
    print("SVM + TFIDF")

    
    for i in range (0,pl_loop):
        print(">>" + str(i))
        X_train, X_test, y_train, y_test = train_test_split(df_tfidf, df_bow_target_parecer, test_size=0.2)
        
        clf = svm.SVC(kernel='linear', C=100)

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
    
    #-------------------------------------
    print("LSTM + Word2Vec")
    
    df = df[ (df.parecer == 'D') | (df.parecer == 'I')]

    df["texto5"] = df.apply(test,axis=1)

    xxxx=200

    tokenizer = Tokenizer(nb_words=xxxx, lower=True,split=' ')
    tokenizer.fit_on_texts(df['texto5'].values)
    #print(tokenizer.word_index)  # To see the dicstionary
    X = tokenizer.texts_to_sequences(df['texto5'].values)
    X = pad_sequences(X)

    embed_dim = 128
    lstm_out = 300
    batch_size = 32

    

    Y = pd.get_dummies(df['parecer']).values
    #X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  test_size=0.2, seed = 1, stratify=Y)
    X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  test_size=0.2,shuffle=False)

    #Here we train the Network.
    for i in range (0,pl_loop):
        print(">>" + str(i))
        X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,  test_size=0.2)
        
        model = Sequential()
        model.add(Embedding(xxxx, embed_dim,input_length = X.shape[1]))
        model.add(SpatialDropout1D(0.1))
        model.add(LSTM(units=lstm_out, dropout = 0.1, recurrent_dropout = 0.1))
        #model.add(LSTM(units=lstm_out)
        ##model.add(Dropout(0.2))
        model.add(Dense(2,activation='softmax'))
        model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
        print(model.summary())
        
        model.fit(X_train, Y_train)

        Y_pred = model.predict(X_valid)

        y_pred = binarize(Y_pred)
        y_true = binarize(Y_valid)

        print(classification_report(y_true, y_pred))

## Base de dados inteira

In [7]:
df_path = 'E:/Code/prf-recursos-multas/datasets/multas_v3.csv'
df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

In [8]:
df_bow = gerar_bow_df(df)

In [9]:
df_tfidf = gerar_tfidf_df(df_bow)

In [ ]:
pipeline(df, df_bow, df_tfidf)

Naive-bayes + BOW
>>0
Accuracy: 0.8695652173913043
              precision    recall  f1-score   support

           0       0.88      0.97      0.93       192
           1       0.72      0.34      0.46        38

    accuracy                           0.87       230
   macro avg       0.80      0.66      0.70       230
weighted avg       0.86      0.87      0.85       230

>>1
Accuracy: 0.8739130434782608
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       194
           1       0.71      0.33      0.45        36

    accuracy                           0.87       230
   macro avg       0.80      0.65      0.69       230
weighted avg       0.86      0.87      0.85       230

>>2
Accuracy: 0.8695652173913043
              precision    recall  f1-score   support

           0       0.89      0.96      0.93       198
           1       0.56      0.28      0.38        32

    accuracy                           0.87       230
   macro av

Accuracy: 0.8652173913043478
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       200
           1       0.48      0.43      0.46        30

    accuracy                           0.87       230
   macro avg       0.70      0.68      0.69       230
weighted avg       0.86      0.87      0.86       230

>>4
Accuracy: 0.8608695652173913
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       198
           1       0.50      0.53      0.52        32

    accuracy                           0.86       230
   macro avg       0.71      0.72      0.72       230
weighted avg       0.86      0.86      0.86       230

>>5
Accuracy: 0.8608695652173913
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       191
           1       0.60      0.54      0.57        39

    accuracy                           0.86       230
   macro avg       0.75      0.73

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95       207
           1       0.00      0.00      0.00        23

    accuracy                           0.90       230
   macro avg       0.45      0.50      0.47       230
weighted avg       0.81      0.90      0.85       230

>>1
Accuracy: 0.8913043478260869


C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

>>0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3786, 128)         25600     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 3786, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               514800    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None
29/29 [==============================] - 1781s 61s/step - loss: 0.5541 - accuracy: 0.8004
              precision    recall  f1-score   support

           0       1.00      0.19      0.33        36
           1       0.87      1.00      0

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.85      1.00      0.92       192

    accuracy                           0.85       227
   macro avg       0.42      0.50      0.46       227
weighted avg       0.72      0.85      0.78       227

>>6
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3786, 128)         25600     
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 3786, 128)         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 300)               514800    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-t

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 3786, 128)         25600     
_________________________________________________________________
spatial_dropout1d_12 (Spatia (None, 3786, 128)         0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None
29/29 [==============================] - 1781s 61s/step - loss: 0.5365 - accuracy: 0.7904
              precision    recall  f1-score   support

           0       1.00      0.23      0.37        31
           1       0.89      1.00      0.

29/29 [==============================] - 1782s 60s/step - loss: 0.5207 - accuracy: 0.7890
              precision    recall  f1-score   support

           0       1.00      0.20      0.33        30
           1       0.89      1.00      0.94       197

    accuracy                           0.89       227
   macro avg       0.95      0.60      0.64       227
weighted avg       0.91      0.89      0.86       227

>>19
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 3786, 128)         25600     
_________________________________________________________________
spatial_dropout1d_19 (Spatia (None, 3786, 128)         0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_19 (Dense)           

## Meio a Meio

In [8]:
df_path = 'E:/Code/prf-recursos-multas/datasets/multas_v3.csv'
df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

dfi = df[ df['parecer'] == 'I' ][0:173]
dfd = df[ df['parecer'] == 'D' ]

df2 = pd.concat([dfi,dfd])
df = df2.sample(frac=1, random_state=1).reset_index(drop=True)

df_bow = gerar_bow_df(df)
df_tfidf = gerar_tfidf_df(df_bow)

pipeline(df, df_bow, df_tfidf)

Naive-bayes + BOW
>>0
Accuracy: 0.4857142857142857
              precision    recall  f1-score   support

           0       0.54      0.54      0.54        39
           1       0.42      0.42      0.42        31

    accuracy                           0.49        70
   macro avg       0.48      0.48      0.48        70
weighted avg       0.49      0.49      0.49        70

>>1
Accuracy: 0.5142857142857142
              precision    recall  f1-score   support

           0       0.50      0.65      0.56        34
           1       0.54      0.39      0.45        36

    accuracy                           0.51        70
   macro avg       0.52      0.52      0.51        70
weighted avg       0.52      0.51      0.51        70

>>2
Accuracy: 0.7
              precision    recall  f1-score   support

           0       0.68      0.77      0.72        35
           1       0.73      0.63      0.68        35

    accuracy                           0.70        70
   macro avg       0.70   

Accuracy: 0.6
              precision    recall  f1-score   support

           0       0.65      0.54      0.59        37
           1       0.56      0.67      0.61        33

    accuracy                           0.60        70
   macro avg       0.60      0.60      0.60        70
weighted avg       0.61      0.60      0.60        70

>>4
Accuracy: 0.7
              precision    recall  f1-score   support

           0       0.74      0.64      0.69        36
           1       0.67      0.76      0.71        34

    accuracy                           0.70        70
   macro avg       0.70      0.70      0.70        70
weighted avg       0.71      0.70      0.70        70

>>5
Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.82      0.66      0.73        41
           1       0.62      0.79      0.70        29

    accuracy                           0.71        70
   macro avg       0.72      0.73      0.71        70
weighted 

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4857142857142857
              precision    recall  f1-score   support

           0       0.45      1.00      0.62        30
           1       1.00      0.10      0.18        40

    accuracy                           0.49        70
   macro avg       0.73      0.55      0.40        70
weighted avg       0.77      0.49      0.37        70

>>2
Accuracy: 0.5714285714285714
              precision    recall  f1-score   support

           0       0.75      0.32      0.44        38
           1       0.52      0.88      0.65        32

    accuracy                           0.57        70
   macro avg       0.63      0.60      0.55        70
weighted avg       0.64      0.57      0.54        70

>>3
Accuracy: 0.4714285714285714
              precision    recall  f1-score   support

           0       0.53      0.26      0.35        38
           1       0.45      0.72      0.55        32

    accuracy                           0.47        70
   macro avg       0.49      0.49

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


>>0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3395, 128)         25600     
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 3395, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               514800    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None
9/9 [==============================] - 475s 54s/step - loss: 0.6752 - accuracy: 0.6308
              precision    recall  f1-score   support

           0       0.76      0.34      0.47        38
           1       0.53      0.88      0.66

9/9 [==============================] - 448s 50s/step - loss: 0.6802 - accuracy: 0.5867
              precision    recall  f1-score   support

           0       0.86      0.56      0.68        34
           1       0.69      0.92      0.79        36

    accuracy                           0.74        70
   macro avg       0.78      0.74      0.73        70
weighted avg       0.77      0.74      0.73        70

>>10
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 3395, 128)         25600     
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 3395, 128)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_10 (Dense)             (

9/9 [==============================] - 545s 59s/step - loss: 0.6789 - accuracy: 0.5966
              precision    recall  f1-score   support

           0       0.53      0.89      0.67        35
           1       0.67      0.23      0.34        35

    accuracy                           0.56        70
   macro avg       0.60      0.56      0.50        70
weighted avg       0.60      0.56      0.50        70

>>14
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 3395, 128)         25600     
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 3395, 128)         0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_14 (Dense)             (

9/9 [==============================] - 530s 59s/step - loss: 0.6785 - accuracy: 0.6108
              precision    recall  f1-score   support

           0       0.63      0.76      0.69        25
           1       0.85      0.76      0.80        45

    accuracy                           0.76        70
   macro avg       0.74      0.76      0.75        70
weighted avg       0.77      0.76      0.76        70

>>18
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 3395, 128)         25600     
_________________________________________________________________
spatial_dropout1d_18 (Spatia (None, 3395, 128)         0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_18 (Dense)             (

## 2 para 1

In [9]:
df_path = 'E:/Code/prf-recursos-multas/datasets/multas_v3.csv'
df = pd.read_csv(df_path, encoding='utf-8',converters={"texto3": eval, "texto4": eval})

dfi = df[ df['parecer'] == 'I' ][0:173*2]
dfd = df[ df['parecer'] == 'D' ]
df2 = pd.concat([dfi,dfd])
df = df2.sample(frac=1, random_state=1).reset_index(drop=True)

df_bow = gerar_bow_df(df)
df_tfidf = gerar_tfidf_df(df_bow)

pipeline(df, df_bow, df_tfidf)

Naive-bayes + BOW
>>0
Accuracy: 0.7980769230769231
              precision    recall  f1-score   support

           0       0.79      0.94      0.86        67
           1       0.83      0.54      0.66        37

    accuracy                           0.80       104
   macro avg       0.81      0.74      0.76       104
weighted avg       0.80      0.80      0.79       104

>>1
Accuracy: 0.7211538461538461
              precision    recall  f1-score   support

           0       0.72      0.91      0.81        66
           1       0.71      0.39      0.51        38

    accuracy                           0.72       104
   macro avg       0.72      0.65      0.66       104
weighted avg       0.72      0.72      0.70       104

>>2
Accuracy: 0.7115384615384616
              precision    recall  f1-score   support

           0       0.71      0.92      0.80        65
           1       0.74      0.36      0.48        39

    accuracy                           0.71       104
   macro av

Accuracy: 0.7596153846153846
              precision    recall  f1-score   support

           0       0.81      0.83      0.82        69
           1       0.65      0.63      0.64        35

    accuracy                           0.76       104
   macro avg       0.73      0.73      0.73       104
weighted avg       0.76      0.76      0.76       104

>>4
Accuracy: 0.7307692307692307
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        71
           1       0.58      0.58      0.58        33

    accuracy                           0.73       104
   macro avg       0.69      0.69      0.69       104
weighted avg       0.73      0.73      0.73       104

>>5
Accuracy: 0.7019230769230769
              precision    recall  f1-score   support

           0       0.72      0.88      0.79        68
           1       0.62      0.36      0.46        36

    accuracy                           0.70       104
   macro avg       0.67      0.62

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.68      1.00      0.81        71
           1       0.00      0.00      0.00        33

    accuracy                           0.68       104
   macro avg       0.34      0.50      0.41       104
weighted avg       0.47      0.68      0.55       104

>>1
Accuracy: 0.6538461538461539


C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\Felipe\anaconda3\envs\novo\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


>>0
Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 3433, 128)         25600     
_________________________________________________________________
spatial_dropout1d_20 (Spatia (None, 3433, 128)         0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None
13/13 [==============================] - 808s 62s/step - loss: 0.6772 - accuracy: 0.6507
              precision    recall  f1-score   support

           0       1.00      0.17      0.29        29
           1       0.76      1.00     

13/13 [==============================] - 819s 64s/step - loss: 0.6661 - accuracy: 0.6496
              precision    recall  f1-score   support

           0       0.95      0.45      0.61        40
           1       0.74      0.98      0.85        64

    accuracy                           0.78       104
   macro avg       0.84      0.72      0.73       104
weighted avg       0.82      0.78      0.76       104

>>6
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 3433, 128)         25600     
_________________________________________________________________
spatial_dropout1d_26 (Spatia (None, 3433, 128)         0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_26 (Dense)             

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 3433, 128)         25600     
_________________________________________________________________
spatial_dropout1d_32 (Spatia (None, 3433, 128)         0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 602       
Total params: 541,002
Trainable params: 541,002
Non-trainable params: 0
_________________________________________________________________
None
13/13 [==============================] - 810s 63s/step - loss: 0.6499 - accuracy: 0.6676
              precision    recall  f1-score   support

           0       1.00      0.21      0.34        29
           1       0.77      1.00      0.8

13/13 [==============================] - 845s 65s/step - loss: 0.7404 - accuracy: 0.5631
              precision    recall  f1-score   support

           0       1.00      0.22      0.36        37
           1       0.70      1.00      0.82        67

    accuracy                           0.72       104
   macro avg       0.85      0.61      0.59       104
weighted avg       0.81      0.72      0.66       104

>>19
Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 3433, 128)         25600     
_________________________________________________________________
spatial_dropout1d_39 (Spatia (None, 3433, 128)         0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 300)               514800    
_________________________________________________________________
dense_39 (Dense)            